# ShoeFly Ad campaign data analysis project 

### Overview

ShoeFly is a global clothing store who recently lunched an online campaign to promote their new promotional products. The scope of this project is to clean and calculate new columns based on the data provided by the marketing team, which consists of a CSV file with the following information: <br>

1. user_id = hash unique number representing user 
2. utm_source = social media platform
3. day = day name and number when the ad was clicked
4. ad_click_timestamp = time when user clicked the ad, format hh:mm
5. experimental_group = target group

In [2]:
import pandas as pd
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Part 1: Data wrangling

In [3]:
script_path = os.path.curdir #directory where the script is run.
rel_path = "ad_clicks.txt"
filepath = os.path.join(script_path, rel_path) #create file path

In [4]:
ad_clicks = pd.read_csv(filepath) # store df in a file

**1. Inspect the first 10 lines of the dataset**

In [6]:
print(ad_clicks.head(10), '\n')
print(ad_clicks.columns.tolist()) # print all the headers of this file

                                user_id utm_source            day ad_click_timestamp experimental_group
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday               7:18                  A
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday                NaN                  B
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday                NaN                  A
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday                NaN                  B
4  012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday                NaN                  B
5  013b0072-7b72-40e7-b698-98b4d0c9967f   facebook     1 - Monday                NaN                  A
6  0153d85b-7660-4c39-92eb-1e1acd023280     google   4 - Thursday                NaN                  A
7  01555297-d6e6-49ae-aeba-1b196fdbb09f     google  3 - Wednesday                NaN                  A
8  018cea61-19ea-4119-895b-1a4309ccb148      email     1 - Monda

**2. Count the number of users targeted per social media platform**

In [8]:
views_per_utm_source = ad_clicks.groupby('utm_source').user_id.count().reset_index()
views_per_utm_source.rename(columns={'user_id':'count_user'}, inplace=True)

print(views_per_utm_source)

  utm_source  count_user
0      email         255
1   facebook         504
2     google         680
3    twitter         215


**3. Add a new column 'day number' based on 'day' column and delete everything before day name in each cell**

In [45]:
ad_clicks['day_nr'] = ad_clicks['day'].str.split('-').str[0] #add new column
ad_clicks['day'] = ad_clicks['day'].str.split(' - ').str[-1] #get the day name
print(ad_clicks.head(10))

                                user_id utm_source         day ad_click_timestamp experimental_group      day_nr  is_click
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google    Saturday               7:18                  A    Saturday     False
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook      Sunday                NaN                  B      Sunday      True
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter     Tuesday                NaN                  A     Tuesday      True
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google     Tuesday                NaN                  B     Tuesday      True
4  012137e6-7ae7-4649-af68-205b4702169c   facebook      Sunday                NaN                  B      Sunday      True
5  013b0072-7b72-40e7-b698-98b4d0c9967f   facebook      Monday                NaN                  A      Monday      True
6  0153d85b-7660-4c39-92eb-1e1acd023280     google    Thursday                NaN                  A    Thursday      True
7  01555297-d6e6

**4. Create a new column called 'is_click' to check the nr. of users who clicked the ad. If 'ad_click_timestamp' is not null, people checked the add, if null, they did not.**

In [46]:
ad_clicks['is_click'] = ad_clicks.ad_click_timestamp.isnull()

**5. Marketing team wants to know the % of people who clicked on ads per platform.<br>
First, group by 'utm_source' and 'is_click' and count the total nr. of users.**

In [51]:
clicks_by_source = ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()
clicks_by_source.rename(columns={'user_id':'count_users'}, inplace=True) # change user_id column name

print(clicks_by_source, '\n')

  utm_source  is_click  count_users
0      email     False           80
1      email      True          175
2   facebook     False          180
3   facebook      True          324
4     google     False          239
5     google      True          441
6    twitter     False           66
7    twitter      True          149 



**6. Create a new dataframe with rows as 'utm_source'and columns "is_click" to show the the number of users who clicked/not clicked. <br>**

In [52]:
clicks_pivot = clicks_by_source.pivot(columns='is_click', index='utm_source', values='count_users').reset_index()
print(clicks_pivot)

is_click utm_source  False  True
0             email     80   175
1          facebook    180   324
2            google    239   441
3           twitter     66   149


**7. Add a new column to "clicks_pivot" to show the percentage of users who clicked the ad, per platform.**

In [55]:
clicks_pivot['percent_clicked'] = round((clicks_pivot[True]/(clicks_pivot[True]+clicks_pivot[False]) * 100), 2)
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email     80   175            68.63
1          facebook    180   324            64.29
2            google    239   441            64.85
3           twitter     66   149            69.30


### Part 2: Analyzing an A/B Test

**8. Create a new table to show the nr. of users pertaining to group A and B.**

In [56]:
check_experimental = ad_clicks.groupby('experimental_group').is_click.count().reset_index()
print(check_experimental)

  experimental_group  is_click
0                  A       827
1                  B       827


**9. Create 2 tables: a_clicks and b_clicks by filtering column 'experimental_group' (target group)**

In [19]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A'].reset_index()
b_clicks = ad_clicks[ad_clicks.experimental_group == "B"].reset_index()
print(a_clicks.head(10), '\n')
print(b_clicks.head(10))

   index                               user_id utm_source         day ad_click_timestamp experimental_group day_nr  is_click
0      0  008b7c6c-7272-471e-b90e-930d548bd8d7     google    Saturday               7:18                  A     6      False
1      2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter     Tuesday                NaN                  A     2       True
2      5  013b0072-7b72-40e7-b698-98b4d0c9967f   facebook      Monday                NaN                  A     1       True
3      6  0153d85b-7660-4c39-92eb-1e1acd023280     google    Thursday                NaN                  A     4       True
4      7  01555297-d6e6-49ae-aeba-1b196fdbb09f     google   Wednesday                NaN                  A     3       True
5      8  018cea61-19ea-4119-895b-1a4309ccb148      email      Monday              18:33                  A     1      False
6     12  01fb228a-9d28-4cde-932c-59b933fa763b      email      Sunday                NaN                  A     7       True


**10. Group the a_clicks and b_clicks by 'day' and 'is_click' and count the total users who clicked or not the add per day. Rename 'user_id' column as 'count_clicks'**

In [44]:
a_clicks_grouped = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()
b_clicks_grouped = b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()

a_clicks_grouped.rename(columns= {'user_id':'count_clicks'},inplace=True)
b_clicks_grouped.rename(columns= {'user_id':'count_clicks'},inplace=True)

print(a_clicks_grouped.head(10), '\n')
print(b_clicks_grouped.head(10))

         day  is_click  count_clicks
0     Friday     False            51
1     Friday      True            77
2     Monday     False            43
3     Monday      True            70
4   Saturday     False            45
5   Saturday      True            73
6     Sunday     False            43
7     Sunday      True            66
8   Thursday     False            47
9   Thursday      True            69 

         day  is_click  count_clicks
0     Friday     False            38
1     Friday      True            90
2     Monday     False            32
3     Monday      True            81
4   Saturday     False            42
5   Saturday      True            76
6     Sunday     False            34
7     Sunday      True            75
8   Thursday     False            29
9   Thursday      True            87


**11. Pivot both tables from above with column 'is_click' as columns and 'count_clicks' as values.**

In [22]:
a_clicks_pivot = a_clicks_grouped.pivot(columns = 'is_click', values ='count_clicks', index ='day').reset_index()
b_clicks_pivot = b_clicks_grouped.pivot(columns = 'is_click', values ='count_clicks', index ='day').reset_index()

**12. Calculate the percentage of people who clicked the ad from total volume of people from that day.**

In [31]:
a_clicks_pivot['percentage_clicked'] = round(a_clicks_pivot[True]/(a_clicks_pivot[False] + a_clicks_pivot[True])*100)
b_clicks_pivot['percentage_clicked'] = round(b_clicks_pivot[True]/(b_clicks_pivot[False] + b_clicks_pivot[True])*100)

#Check final result
print(a_clicks_pivot,"\n")
print(b_clicks_pivot)

is_click         day  False  True  percentage_clicked
0             Friday     51    77                60.0
1             Monday     43    70                62.0
2           Saturday     45    73                62.0
3             Sunday     43    66                61.0
4           Thursday     47    69                59.0
5            Tuesday     43    76                64.0
6          Wednesday     38    86                69.0 

is_click         day  False  True  percentage_clicked
0             Friday     38    90                70.0
1             Monday     32    81                72.0
2           Saturday     42    76                64.0
3             Sunday     34    75                69.0
4           Thursday     29    87                75.0
5            Tuesday     45    74                62.0
6          Wednesday     35    89                72.0


**13. Export pivoted tables in csv files locally**

In [35]:
a_clicks_pivot.to_csv(os.curdir +r"\a_clicks_by_day_final.csv", index=None)
b_clicks_pivot.to_csv(os.curdir + r"\b_clicks_by_day_final.csv", index=None)